## Dependencies

In [1]:
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
model_path = '/kaggle/input/96-googleq-a-train-bert-base-contractions/model.h5'
BERT_PATH = '/kaggle/input/bert-base-from-tfhub/bert_en_uncased_L-12_H-768_A-12'
TOKENIZER_PATH = BERT_PATH+'/assets/vocab.txt'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(TOKENIZER_PATH, True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

bert_layer = KerasLayer(BERT_PATH, trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

x = GlobalAveragePooling1D()(sequence_output)
x = Dropout(0.2)(x)
output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

# Make predictions

In [8]:
Y_test = model.predict(X_test)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.944448,0.743453,0.606825,0.486293,0.378413,0.486317,0.617273,0.625597,0.527251,...,0.945378,0.882616,0.578171,0.975878,0.956516,0.843574,0.023174,0.027883,0.833418,0.934743
1,46,0.932823,0.559478,0.007706,0.697197,0.795906,0.907059,0.548510,0.534279,0.091996,...,0.752807,0.953800,0.641610,0.979511,0.985987,0.854315,0.972436,0.062828,0.045016,0.880804
2,70,0.907577,0.721836,0.074530,0.702573,0.885516,0.916386,0.545230,0.533977,0.124642,...,0.876530,0.880663,0.607892,0.971410,0.944469,0.806233,0.036903,0.081838,0.786076,0.897762
3,132,0.901367,0.467861,0.010374,0.613414,0.810937,0.913004,0.565323,0.561351,0.078953,...,0.773545,0.918971,0.666157,0.968505,0.981237,0.867088,0.834215,0.139570,0.402713,0.893707
4,200,0.926677,0.429084,0.053224,0.831396,0.802159,0.863699,0.635344,0.660955,0.026705,...,0.746285,0.894209,0.647834,0.975639,0.958047,0.816532,0.142631,0.126906,0.379130,0.927057


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.914925,0.629586,0.055295,0.687084,0.801896,0.841523,0.579581,0.527506,0.212780,...,0.839366,0.918383,0.674213,0.969288,0.972742,0.856165,0.520986,0.132254,0.462646,0.912142
std,2812.670060,0.043761,0.143301,0.093730,0.126955,0.113935,0.147779,0.049722,0.086182,0.172322,...,0.076667,0.032090,0.065032,0.011479,0.013862,0.049116,0.357022,0.072818,0.289663,0.024656
min,39.000000,0.709199,0.379749,0.003484,0.203700,0.241441,0.167776,0.442422,0.327978,0.005095,...,0.647812,0.750623,0.521066,0.926555,0.921259,0.691828,0.003648,0.007685,0.007214,0.827506
25%,2572.000000,0.890635,0.497407,0.010374,0.600573,0.771433,0.813699,0.546467,0.468515,0.078246,...,0.776029,0.900264,0.630455,0.962816,0.965827,0.825750,0.115752,0.079459,0.188570,0.896404
50%,5093.000000,0.921691,0.618018,0.017007,0.678785,0.816124,0.899952,0.576220,0.506085,0.157003,...,0.845982,0.922502,0.672522,0.970497,0.975783,0.862045,0.586060,0.123461,0.451897,0.913651
75%,7482.000000,0.948073,0.751969,0.049435,0.781154,0.870619,0.931525,0.611306,0.569275,0.310082,...,0.906614,0.940884,0.715177,0.977402,0.982851,0.889948,0.873049,0.180583,0.687032,0.930272
max,9640.000000,0.987380,0.929043,0.619436,0.984145,0.979007,0.989621,0.738159,0.827080,0.790102,...,0.972411,0.978695,0.873813,0.991922,0.994522,0.961090,0.989082,0.364321,0.989264,0.965857
